# Tarea 2 

# Procesamiento de Lenguaje Natural

# Ernesto Antonio Reyes Ramírez

# 1. Preparación de los datos

In [1]:
import os
import re
from keras.preprocessing.text import Tokenizer

2023-02-13 12:00:35.647516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 12:00:38.374465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-13 12:00:38.374548: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-13 12:00:43.912120: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
def get_texts_from_file(path_corpus,path_truth):
    tr_text = []
    tr_y = []
    
    with open(path_corpus,"r") as f_corpus, open(path_truth,"r") as f_truth:
        for twitt in f_corpus:
            tr_text += [twitt]
        for label in f_truth:
            tr_y += [label]
    return tr_text,tr_y

In [5]:
tr_text,tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

Construcción simple del vocabulario

In [6]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [7]:
import nltk

corpus_palabras = []

for doc in tr_text:
    corpus_palabras += tokenizer.tokenize(doc)
    
fdist = nltk.FreqDist(corpus_palabras)

In [8]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key],key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [9]:
V = sortFreqDict(fdist)

In [78]:
V = V[:5000]

In [11]:
dict_indices = dict()
cont = 0 

for weight, word in V:
    dict_indices[word] = cont
    cont += 1

Términos de validación

In [18]:
val_text,val_y = get_texts_from_file("./mex_val.txt","./mex_val_labels.txt")
val_y = list(map(int,val_y))


from collections import Counter
print(Counter(val_y))

Counter({0: 397, 1: 219})


# 2. Bolsas de palabras, Bigramas y Emociones 

In [20]:
#Cargamos las librerias necesarias para crear y entrenar un modelo de svm
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score, precision_recall_fscore_support, roc_auc_score

tr_y = list(map(int,tr_y))

parameters = {'C':[.05,.12,.25,.5,1,2,4]}

#creamos nuestro modelo
svr = svm.LinearSVC(class_weight="balanced")
grid = GridSearchCV(estimator=svr,param_grid = parameters, n_jobs=4,scoring="f1_macro", cv=5)

 # BoW con pesado binario

In [21]:
#función que crea la BoW para un pesado binario
def build_bow_bi(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=int)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = 1
        cont_doc += 1
    
    return BOW

In [22]:
BOW_tr_bi = build_bow_bi(tr_text,V,dict_indices)

In [24]:
grid.fit(BOW_tr_bi,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [25]:
BOW_val_bi = build_bow_bi(val_text,V,dict_indices)

In [26]:
y_pred = grid.predict(BOW_val_bi)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[329  68]
 [ 47 172]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.85       397
           1       0.72      0.79      0.75       219

    accuracy                           0.81       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.81      0.82       616



# BoW  pesado con frecuencia 

In [27]:
#BoW con pesado frecuencia

def build_bow_fq(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=int)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = fdist_doc[word]
        cont_doc += 1
    
    return BOW

In [108]:
BOW_tr_fq = build_bow_fq(tr_text,V,dict_indices)

In [29]:
grid.fit(BOW_tr_fq,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [30]:
BOW_val_fq = build_bow_fq(val_text,V,dict_indices)

In [31]:
y_pred = grid.predict(BOW_val_fq)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[334  63]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       397
           1       0.73      0.78      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



# BoW con pesado tfidf

Lo que haremos será primero calcular el número de veces que ocurre cada una de las 5000 palabras y
guardarlo en un diccionario, para que al momento de realizar el pesado tfidf sea más rápido

In [101]:
vec_df = []
for tr in tr_text:
    vec_df.append(tokenizer.tokenize(tr))
    
df = dict()
for aux, word in V:
    cont = 0
    for tr in vec_df:
        if word in tr:
            cont += 1
    df[word] = cont

In [104]:
def build_bow_tfidf(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = fdist_doc[word]*np.log(len(tr_text)/df[word])
        cont_doc += 1
    
    return BOW

In [106]:
BOW_tr_tfidf = build_bow_tfidf(tr_text,V,dict_indices)

In [109]:
grid.fit(BOW_tr_tfidf,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [110]:
BOW_val_tfidf = build_bow_tfidf(val_text,V,dict_indices)

In [111]:
y_pred = grid.predict(BOW_val_tfidf)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[368  29]
 [161  58]]
              precision    recall  f1-score   support

           0       0.70      0.93      0.79       397
           1       0.67      0.26      0.38       219

    accuracy                           0.69       616
   macro avg       0.68      0.60      0.59       616
weighted avg       0.69      0.69      0.65       616



# BoW con pesado binario normalizado l2

In [112]:
#Función que crea la BoW para un pesado binario
def build_bow_bn(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = 1
        cont_doc += 1
        
    #Calculamos las normas
    new_weight = []
    for i in range(len(tr_text)):
        tmp = 0
        for j in range(len(V)):
            tmp = tmp+BOW[i,j]**2
        tmp = np.sqrt(tmp)
        new_weight.append(tmp)
        
    #Actualizamos los pesos
    for i in range(len(tr_text)):
        for j in range(len(V)):
            BOW[i,j] = BOW[i,j]/new_weight[i] 
    
    return BOW

In [114]:
BOW_tr_bn = build_bow_bn(tr_text,V,dict_indices)

In [115]:
grid.fit(BOW_tr_bn,tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [116]:
BOW_val_bn = build_bow_bn(val_text,V,dict_indices)

In [117]:
y_pred = grid.predict(BOW_val_bn)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[323  74]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       397
           1       0.70      0.78      0.73       219

    accuracy                           0.80       616
   macro avg       0.78      0.79      0.79       616
weighted avg       0.81      0.80      0.80       616



# BoW pesado con frecuencia normalizado l2

In [119]:
#BoW pesado con frecuencia normalizado l2

def build_bow_fqn(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = fdist_doc[word]
        cont_doc += 1
    
    #Calculamos las normas
    new_weight = []
    for i in range(len(tr_text)):
        tmp = 0
        for j in range(len(V)):
            tmp = tmp+BOW[i,j]**2
        tmp = np.sqrt(tmp)
        new_weight.append(tmp)
        
    #Actualizamos los pesos
    for i in range(len(tr_text)):
        for j in range(len(V)):
            BOW[i,j] = BOW[i,j]/new_weight[i] 
    
    return BOW

In [120]:
BOW_tr_fqn = build_bow_fqn(tr_text,V,dict_indices)

In [121]:
grid.fit(BOW_tr_fqn,tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [122]:
BOW_val_fqn = build_bow_fqn(val_text,V,dict_indices)

In [123]:
y_pred = grid.predict(BOW_val_fqn)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[321  76]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       397
           1       0.69      0.77      0.73       219

    accuracy                           0.80       616
   macro avg       0.78      0.79      0.78       616
weighted avg       0.80      0.80      0.80       616



# BoW con pesado tfidf normalizado l2

In [125]:
#BoW pesado tdidf normalizado l2

def build_bow_tfidfn(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(V)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                BOW[cont_doc,dict_indices[word]] = fdist_doc[word]*np.log(len(tr_text)/df[word])
        cont_doc += 1
        
    #Calculamos las normas
    new_weight = []
    for i in range(len(tr_text)):
        tmp = 0
        for j in range(len(V)):
            tmp = tmp+BOW[i,j]**2
        tmp = np.sqrt(tmp)
        new_weight.append(tmp)
        
    #Actualizamos los pesos
    for i in range(len(tr_text)):
        for j in range(len(V)):
            BOW[i,j] = BOW[i,j]/new_weight[i] 
    
    return BOW

In [127]:
BOW_tr_tfidfn = build_bow_tfidfn(tr_text,V,dict_indices)

In [128]:
grid.fit(BOW_tr_tfidfn,tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [132]:
BOW_val_tfidfn = build_bow_tfidfn(val_text,V,dict_indices)

In [133]:
y_pred = grid.predict(BOW_val_tfidfn)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[378  19]
 [150  69]]
              precision    recall  f1-score   support

           0       0.72      0.95      0.82       397
           1       0.78      0.32      0.45       219

    accuracy                           0.73       616
   macro avg       0.75      0.63      0.63       616
weighted avg       0.74      0.73      0.69       616



# Tabla comparativa

|  Pesado  | F1 score de la etiqueta 1| Accuracy  | macro avg
|-----------|----------|------------- | -----------
| Binario |  0.75  |  0.81    | 0.80
| Frecuencia |  0.75  |  0.82 |  0.80
| TfIdf |  0.38  |  0.69   | 0.59
| Binario Norma l2  | 0.73  |  0.80   | 0.79
| Frecuencia Norma l2  | 0.73  |   0.80| 0.78
| TfIdf Norma l2 |  0.45  |  0.73|   0.63

De las configuraciones anteriores elija la mejor y evalúela con más y menos términos
(e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.

Sol: Vamos a tomar como mejor modelo el que utiliza el pesado con frecuencia y trabajar sobre él. 

In [140]:
#7000 palabras más frecuentes
W = sortFreqDict(fdist)
A = W[:7000]

In [141]:
dict_indices1 = dict()
cont = 0 

for weight, word in A:
    dict_indices1[word] = cont
    cont += 1

In [142]:
BOW_fq_7000 = build_bow_fq(tr_text,A,dict_indices1)

In [143]:
grid.fit(BOW_fq_7000,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [144]:
BOW_val_fq_7000 = build_bow_fq(val_text,A,dict_indices1)

In [145]:
y_pred = grid.predict(BOW_val_fq_7000)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[334  63]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       397
           1       0.73      0.78      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



In [153]:
#1000 más usados 
E = W[:1000]

In [147]:
dict_indices2 = dict()
cont = 0 

for weight, word in E:
    dict_indices2[word] = cont
    cont += 1

In [148]:
BOW_fq_1000 = build_bow_fq(tr_text,E,dict_indices2)

In [149]:
grid.fit(BOW_fq_1000,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [150]:
BOW_val_fq_1000 = build_bow_fq(val_text,E,dict_indices2)

In [151]:
y_pred = grid.predict(BOW_val_fq_1000)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[336  61]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       397
           1       0.74      0.78      0.76       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.81       616
weighted avg       0.82      0.82      0.82       616



# Tabla comparativa entre distintas poblaciones del pesado con frecuencia

|  No. de términos  | F1 score de la etiqueta 1| Accuracy  | macro avg
|-----------|----------|------------- | -----------
| 7000 |  0.75  |  0.81    | 0.80
| 5000 |  0.75  |  0.82 |  0.80
| 1000 |  0.38  |  0.69   | 0.59

# Bolsa de Emociones con EmoLex

In [167]:
table = []
with open("./Spanish-NRC-EmoLex.txt","r") as f:
        for emotion in f:
            table += [tokenizer.tokenize(emotion)]

In [168]:
emotions = ['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust',]

In [170]:
table = table[1:]

In [172]:
table = [p for p in table if len(p)==12]

In [196]:
valores = dict()

for word in table:
    aux = []
    for i in range(1,11):
        aux.append(int(word[i]))
    valores[word[11]] = aux 

In [198]:
def build_boe(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(emotions)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                if word in valores:
                    for i in range(0,10):
                        BOW[cont_doc,i] = BOW[cont_doc,i] + fdist_doc[word]*valores[word][i]
        cont_doc += 1
    
    return BOW

In [199]:
BOE_fq = build_boe(tr_text,V,dict_indices)

In [280]:
#Aqui tenemos la bolsa de emociones
BOE_fq

array([[0., 2., 0., ..., 1., 0., 0.],
       [0., 2., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 2.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]])

# Evalúa tu BoE clasificando con SVM. 

In [202]:
#Pesado con frecuencia 
grid.fit(BOE_fq,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [203]:
BOE_val_fq = build_boe(val_text,V,dict_indices)

In [204]:
y_pred = grid.predict(BOE_val_fq)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[197 200]
 [ 77 142]]
              precision    recall  f1-score   support

           0       0.72      0.50      0.59       397
           1       0.42      0.65      0.51       219

    accuracy                           0.55       616
   macro avg       0.57      0.57      0.55       616
weighted avg       0.61      0.55      0.56       616



In [205]:
#pesado binario
def build_boe_bi(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),len(emotions)),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                if word in valores:
                    for i in range(0,10):
                        if BOW[cont_doc,i] == 0:
                            BOW[cont_doc,i] = BOW[cont_doc,i] + valores[word][i]
        cont_doc += 1
    
    return BOW

In [206]:
BOE_bi = build_boe_bi(tr_text,V,dict_indices)

In [208]:
BOE_bi

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [209]:
#Pesado binario
grid.fit(BOE_bi,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [210]:
BOE_val_bi = build_boe_bi(val_text,V,dict_indices)

In [211]:
y_pred = grid.predict(BOE_val_bi)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[181 216]
 [ 69 150]]
              precision    recall  f1-score   support

           0       0.72      0.46      0.56       397
           1       0.41      0.68      0.51       219

    accuracy                           0.54       616
   macro avg       0.57      0.57      0.54       616
weighted avg       0.61      0.54      0.54       616



Primero calculamos el número de documentos en los que aparecen las emociones y lo guardamos 
en un vector para hacer todo más rápido

In [222]:
mat = [0]*10

for i in range(0,10):
    for tr in vec_df:
        cont = 0
        for word in tr:
            if word in valores and valores[word][i] == 1:
                cont = 1
                break
        mat[i] = mat[i] + cont

In [224]:
# BOW pesado tfidf
def build_boe_ti(tr_text,V,dict_indices):
    BOW = build_boe(tr_text,V,dict_indices)
    
    for i in range(len(tr_text)):
        for j in range(len(emotions)):
            BOW[i][j] = BOW[i][j]*np.log(len(tr_text)/mat[j])
    
    return BOW

In [225]:
BOE_ti = build_boe_ti(tr_text,V,dict_indices)

In [226]:
BOE_ti

array([[0.        , 1.41463271, 0.        , ..., 0.93064287, 0.        ,
        0.        ],
       [0.        , 1.41463271, 0.        , ..., 0.93064287, 0.        ,
        0.        ],
       [0.        , 0.70731635, 0.        , ..., 0.93064287, 0.        ,
        0.98275511],
       ...,
       [0.        , 0.70731635, 0.        , ..., 0.93064287, 0.        ,
        1.96551022],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.70731635, 0.        , ..., 0.        , 0.        ,
        0.98275511]])

In [227]:
grid.fit(BOE_ti,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [228]:
BOE_val_ti = build_boe_ti(val_text,V,dict_indices)

In [229]:
y_pred = grid.predict(BOE_val_ti)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[ 45 352]
 [ 32 187]]
              precision    recall  f1-score   support

           0       0.58      0.11      0.19       397
           1       0.35      0.85      0.49       219

    accuracy                           0.38       616
   macro avg       0.47      0.48      0.34       616
weighted avg       0.50      0.38      0.30       616



|  Pesado  | F1 score de la etiqueta 1| Accuracy  | macro avg
|-----------|----------|------------- | -----------
| Binario |  0.51  |  0.54    | 0.54
| Frecuencia |  0.51  |  0.55 |  0.55
| TfIdf |  0.49  |  0.38   | 0.34

# 3. Recurso Linguístico de Emociones Mexicano 

In [239]:
doc = []
with open("./SEL.txt","r",encoding='latin-1') as f:
        for emotion in f:
            doc += [tokenizer.tokenize(emotion)]

In [242]:
doc = [p for p in doc if len(p)==3]

In [265]:
doc = doc[1:]

In [275]:
emociones = {'Alegría':0,'Enojo':1,'Miedo':2, 'Repulsión':3,'Sorpresa':4,'Tristeza':5}

In [273]:
dicc = dict()

for word in doc:
    dicc[word[0]] = [0,0,0,0,0,0]
    
for word in doc:
    dicc[word[0]][emociones[word[2]]] = 1  

In [277]:
def build_boe_fq(tr_text,V,dict_indices):
    BOW = np.zeros((len(tr_text),6),dtype=float)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_indices:
                if word in dicc:
                    for i in range(0,6):
                        BOW[cont_doc,i] = BOW[cont_doc,i] + fdist_doc[word]*dicc[word][i]
        cont_doc += 1
    
    return BOW

In [278]:
BOE_fq_sel = build_boe_fq(tr_text,V,dict_indices)

In [279]:
#Bolsa de palabras
BOE_fq_sel

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [281]:
grid.fit(BOE_fq_sel,tr_y)

/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarni

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [282]:
BOE_val_fq_sel = build_boe_fq(val_text,V,dict_indices)

In [283]:
y_pred = grid.predict(BOE_val_fq_sel)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[352  45]
 [182  37]]
              precision    recall  f1-score   support

           0       0.66      0.89      0.76       397
           1       0.45      0.17      0.25       219

    accuracy                           0.63       616
   macro avg       0.56      0.53      0.50       616
weighted avg       0.59      0.63      0.57       616



En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el
"Probability Factor of Affective use". No más de 5 renglones.


Sol: No lo implementé pero tenía la idea agregarlo como unn factor de crecimiento alpha que multiplica la frecuencia de la palabra, con esto estaríamos controlando más el nivel del sentimiento que queremos capturar. Si el PFA es cercano a uno entonces al multiplicar a la frecuencia será casi igual o disminuirá, pero si el PFA es cercano a cero hará muy pequeña la contribución de la palabra, que es lo que esperaríamos ya que si no se relaciona tanto a ese sentimiento no debería contar tanto. 

# 4. ¿Podemos mejorar con bigramas?

Hacer un experimento dónde concatene una buena BoW según sus experimentos anteri-
ores con otra BoW construida a partir de los 1000 bigramas más frecuentes.

In [292]:
#Vamos a considerar la BoW con pesado que utiliza la frecuencia ya que nos dió los mejores resultados
BOW_tr_fq

array([[1, 2, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [293]:
from nltk.util import bigrams

In [297]:
#Utilizaré el siguiente corpus que se realizó en un inicio

bigra = list(bigrams(corpus_palabras))

In [298]:
bigra

[('lo', 'peor'),
 ('peor', 'de'),
 ('de', 'todo'),
 ('todo', 'es'),
 ('es', 'que'),
 ('que', 'no'),
 ('no', 'me'),
 ('me', 'dan'),
 ('dan', 'por'),
 ('por', 'un'),
 ('un', 'tiempo'),
 ('tiempo', 'y'),
 ('y', 'luego'),
 ('luego', 'vuelven'),
 ('vuelven', 'estoy'),
 ('estoy', 'hasta'),
 ('hasta', 'la'),
 ('la', 'verga'),
 ('verga', 'de'),
 ('de', 'estl'),
 ('estl', 'a'),
 ('a', 'la'),
 ('la', 'vga'),
 ('vga', 'no'),
 ('no', 'seas'),
 ('seas', 'mamón'),
 ('mamón', '45'),
 ('45', 'putos'),
 ('putos', 'minutos'),
 ('minutos', 'después'),
 ('después', 'me'),
 ('me', 'dices'),
 ('dices', 'que'),
 ('que', 'apenas'),
 ('apenas', 'sales'),
 ('sales', 'no'),
 ('no', 'me'),
 ('me', 'querías'),
 ('querías', 'avisar'),
 ('avisar', 'en'),
 ('en', '3'),
 ('3', 'horas'),
 ('horas', '?'),
 ('?', '😑'),
 ('😑', 'considero'),
 ('considero', 'que'),
 ('que', 'lo'),
 ('lo', 'más'),
 ('más', 'conveniente'),
 ('conveniente', 'seria'),
 ('seria', 'que'),
 ('que', 'lo'),
 ('lo', 'retes'),
 ('retes', 'a'),
 ('a', 

In [299]:
fdistb = nltk.FreqDist(bigra)

In [300]:
fdistb

FreqDist({('!', '!'): 591, ('la', 'verga'): 419, ('a', 'la'): 391, ('de', 'la'): 311, ('@usuario', '@usuario'): 298, ('que', 'no'): 221, ('que', 'me'): 220, ('la', 'madre'): 194, ('los', 'putos'): 175, ('en', 'la'): 174, ...})

In [301]:
K = sortFreqDict(fdistb)

In [302]:
K = K[:1000]

In [304]:
dict_b = dict()
cont = 0 

for weight, word in K:
    dict_b[word] = cont
    cont += 1

In [307]:
#BoW con pesado frecuencia para bigramas

def build_bow_fq_b(tr_text,K,dict_b):
    BOW = np.zeros((len(tr_text),len(K)),dtype=int)
    cont_doc = 0
    for tr in tr_text:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_b:
                BOW[cont_doc,dict_b[word]] = fdist_doc[word]
        cont_doc += 1
    
    return BOW

In [308]:
BOW_bi_fq = build_bow_fq_b(tr_text,K,dict_b)

In [310]:
grid.fit(BOW_bi_fq,tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=4,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [311]:
BOW_val_fq_bi = build_bow_fq_b(val_text,K,dict_b)

In [312]:
y_pred = grid.predict(BOW_val_fq_bi)

p, r, f, _ = precision_recall_fscore_support(val_y, y_pred, average="macro", pos_label=1)

print(confusion_matrix(val_y,y_pred))
print(metrics.classification_report(val_y,y_pred))

[[397   0]
 [219   0]]
              precision    recall  f1-score   support

           0       0.64      1.00      0.78       397
           1       0.00      0.00      0.00       219

    accuracy                           0.64       616
   macro avg       0.32      0.50      0.39       616
weighted avg       0.42      0.64      0.51       616



/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ernesto/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ernesto/anaconda3/lib/

Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi-
gramas; usted elige las dimensionalidades. Para construir la representación final del

documento utilice la concatenación de las representaciones según sus observaciones
(e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa
de Sentimientos de Grigori), y aliméntelas a un SVM.

Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y
posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras
a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también
brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea
breve: todo en NO más de dos párrafos.

# Conclusiones

En el punto 1 podemos notar que de entre los 6 modelos de pesados que utilizamos, los que mejores resultados mostrarón fueron los de pesado con frecuencia y pesado binario, con valores de accurary entre 0.81 y 0.82. Por otro lado mientras el pesado tfidf obtuvo resultados mucho más bajo, esto se debe (creo) a que estamos analizando tweets, las oraciones son muy pequeñas y utilizan muy pocos términos por lo que el factor idf casi no penaliza. Este tipo de pesado se esperaría para textos muy muy grandes y una gran cantidad de ellas. Pudimos notar que con la normalización los valores para los pesados binarios y de frecuencia se mantuvieron y para los de tfidf se incrementó un poco. Al variar el número de palabras a considerar obtuvimos mejores resultados con solo 1000 palabras, esto se debe a que estas deben ser las que dsicriminan mejor si un tweet es agresivo o no, y las siguientes pueden ser palabras que se repiten mucho pero no discriminan. 

Utilizando bolsa de emociones obtuvimos resultados más bajos que con la bolsa de palabras. A mi entender, esto es lo que se debería esperar ya que estamos representando nuestro documento con un vector de dimensión muchísima menor, al restringirnos a las emociones. De modo que estamos capturando unn número menor de carácterísticas. En ese caso el pesado que mejor se comportó fue el de frecuencia, y es que a comparación del binario si tuvieramos en un tweet una palabra mala que se repite mucho solo sería capturada una vez, mientras que para el de frecuencia se caputraría su podera contribución al repetirse mucho. Finalmente con la bolsa de palabras se obtienen resultados parecidos o un poco más bajo.